In [109]:
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:
df = pd.read_csv('df.csv')

In [111]:
from dotenv import load_dotenv
import os

load_dotenv()

open_ai_key = os.getenv("OPEN_AI_KEY")
google_search_key = os.getenv("GOOGLE_PROGRAMMABLE_SEARCH_KEY")


In [112]:
len(df)

28682

setup

In [113]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('word2vec_model/GoogleNews-vectors-negative300.bin', binary=True)  

KeyboardInterrupt: 

In [ ]:
from gensim.models.doc2vec import Doc2Vec
doc2vec_model = Doc2Vec.load("doc2vec_model/doc2vec_wiki_d300_n5_w8_mc50_t12_e10_dbow.model")

In [ ]:
import nltk
from nltk import ngrams, pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter

def generate_ngram_results(set_ids, df, count_threshold = 3):
    # Initialize dictionary to store results
    ngram_results = {i: [] for i in set_ids}

    # Loop through each EssaySet
    for set_id in set_ids:
        # Filter DataFrame for current EssaySet and score1 == 2
        filtered_df = df[(df['EssaySet'] == set_id) & (df['Score1'] == 2)]
        
        # Print progress for each EssaySet
        print(f"Processing EssaySet {set_id} with {len(filtered_df)} essays.")
        
        # Initialize a Counter to count n-grams across all essays in the set
        ngram_counter = Counter()
        
        # Process each EssayText in the filtered_df DataFrame
        for essay in filtered_df['EssayText']:
            tokens = word_tokenize(essay)
            pos_tags = pos_tag(tokens)
            
            # Generate bi-grams, tri-grams, and tetra-grams
            bi_grams = list(ngrams(pos_tags, 2))
            tri_grams = list(ngrams(pos_tags, 3))
            tetra_grams = list(ngrams(pos_tags, 4))
            
            # Concatenate all n-grams into a single list and update the counter
            all_ngrams = bi_grams + tri_grams + tetra_grams
            ngram_counter.update(all_ngrams)
        
        # Filter n-grams that appeared at least 3 times
        frequent_ngrams = [ngram for ngram, count in ngram_counter.items() if count >= count_threshold]
        
        # Store the frequent n-grams in the results dictionary
        ngram_results[set_id] = frequent_ngrams
        
        # Print after processing each set
        print(f"Completed processing EssaySet {set_id}, with {len(frequent_ngrams)} n-grams.")
    
    return ngram_results


In [ ]:
ngram_results = generate_ngram_results([3], df, 30)

Processing EssaySet 3 with 699 essays.
Completed processing EssaySet 3, with 403 n-grams.


In [ ]:
import os

def load_prompts(set_ids):
    prompts = {}
    for set_id in set_ids:
        file_name = f"prompts/asap_{set_id:02d}.txt"
        if os.path.exists(file_name):
            with open(file_name, 'r') as file:
                prompts[set_id] = file.read().strip()
        else:
            print(f"Prompt file {file_name} not found.")
    return prompts

set_ids = list(range(1, 11))  # Example set ids, you can modify this as needed
prompts = load_prompts(set_ids)


In [126]:
import os

def load_only_prompts(set_ids):
    prompts = {}
    for set_id in set_ids:
        file_name = f"only_prompts/asap_{set_id:02d}.txt"
        if os.path.exists(file_name):
            with open(file_name, 'r') as file:
                prompts[set_id] = file.read().strip()
        else:
            print(f"Prompt file {file_name} not found.")
    return prompts

only_prompts = load_only_prompts(list(range(1, 11)))


In [117]:
import requests
import tiktoken
from bs4 import BeautifulSoup
from openai import OpenAI
client = OpenAI(api_key=open_ai_key)
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
import json
import concurrent.futures

def extract_articles(keyword):
      url = 'https://customsearch.googleapis.com/customsearch/v1'
      params = {
          'key': "",
          'cx': '',
          'q': keyword,
          'start': 1,
          'num': 10,
          'lr': "lang_en",
          "filter": 1
      }      
      articles = []
      all_links = []
      try:
        for i in range(2):
          response = requests.get(url, params={
              **params,
              "start": i * 10 + 1
          })
          data = response.json()
          links = [item['link'] for item in data['items'] if 'link' in item]
          all_links.extend(links)

        print("All links: ", all_links)
        
        # Function to scrape a single link
        def scrape_link(link):
            try:
                page = requests.get(link, timeout=10)
                soup = BeautifulSoup(page.content, 'html.parser')
                return soup.get_text()
            except Exception as e:
                print(f"Failed to scrape {link}: {e}")
                return None

        # Use ThreadPoolExecutor to scrape links concurrently
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = list(executor.map(scrape_link, all_links))

        # Filter out None results and extend articles list
        articles.extend([result for result in results if result is not None])

      except Exception as e:
          print(f"An error occurred: {e}")
          # all.extend(data["items"])
          
      return articles

def extract_domain_specific_keywords(prompt, answers):
    # first 50k tokens
    tokenizer = tiktoken.get_encoding("cl100k_base")
    tokens = tokenizer.encode(answers)
    first_50000_tokens = tokens[:50000]
    extracted_text = tokenizer.decode(first_50000_tokens)

    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": f"Here is a question: {prompt}. Identify 100 domain-specific keywords that are most relevant to the question from the following set of student answers, and return them in a JSON object with an array field named 'keywords':\n\n{extracted_text}"}
        ]
    )
    
    keywords = json.loads(response.choices[0].message.content)

    return keywords

def extract_tfidf_from_articles(keyword, articles):
    def preprocess(text):
        return text.lower()

    # Preprocess articles
    preprocessed_articles = [preprocess(article) for article in articles]

    # Initialize the vectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the articles
    tfidf_matrix = vectorizer.fit_transform(preprocessed_articles)

    # Get the feature names (i.e., the words)
    feature_names = vectorizer.get_feature_names_out()

    # Split the keyword into individual words and lower each
    keywords = keyword.lower().split()

    # Initialize list to store tf-idf values for each word in the keyword
    tfidf_values_list = []

    # Iterate over each word in the keyword
    for word in keywords:
        try:
            # Find the index of the word
            word_index = feature_names.tolist().index(word)
            # Get the tf-idf values for the word across all documents
            tfidf_values = tfidf_matrix[:, word_index].toarray().flatten()
            # Append the average tf-idf value for the word
            tfidf_values_list.append(tfidf_values.mean())
        except ValueError:
            # If the word is not found, append 0.0
            tfidf_values_list.append(0.0)

    # Calculate the average tf-idf value for the entire keyword phrase
    if tfidf_values_list:
        average_tfidf = sum(tfidf_values_list) / len(tfidf_values_list)
    else:
        average_tfidf = 0.0

    return average_tfidf


def extract_weighted_keywords(setnumbers, df, prompts):
  weighted_keywords = {}
  all_articles = {}
  for setno in setnumbers:
    answers = " ".join(df[df['EssaySet'] == setno]['EssayText'].tolist())
    keywords = extract_domain_specific_keywords(prompts[setno], answers)["keywords"]
    # keywords = ['animals', 'Asia', 'Australia', 'Bamboo', 'Bears', 'Biologists', 'Biodiversity', 'Boa constrictor', 'Burmese python', 'Bushnell', 'Cape Coral', 'Carnivores', 'Change', 'China', 'Climates', 'Cobra', 'Cold blooded', 'Creatures', 'Customer', 'Ecosystem', 'Ecophobia', 'Effective threat', 'Endangered species', 'Environment', 'Everglades', 'Exclusive food', 'Exploded', 'Food source', 'Florida', 'Food stability', 'Food variety', 'Generalists', 'Government', 'Habitat', 'Hankering', 'Herbivores', 'Hood spread', 'Households', 'Humans', 'Hurricane Andrew', 'Invasive species', 'Key Largo wood rat', 'Kimberly', 'Koala', 'Lizards', 'Macroherbivores', 'Macinnes', 'Madagascar', 'Mammals', 'Mcinnes', 'Microchip', 'Monitor lizard', 'Natural predators', 'Niche environments', 'Nile monitor lizard', 'Non-native', 'Organisms', 'Panda', 'Pandas', 'Permit', 'Pets', 'Plants', 'Policy adviser', 'Potomac', 'Predators', 'Prey', 'Python', 'Raccoon', 'Radiated tortoises', 'Reed', 'Reptiles', 'Resource-specific', 'RobRoy Macinnes', 'Scales', 'Scorpion', 'Sensitive habitats', 'Skip Snow', 'Specialists', 'Stability', 'Survival', 'Tourists', 'Traditional diet', 'Urban areas', 'U.S. Fish and Wildlife Service', 'U.S. Geological Survey', 'Unique habitat', 'Variety of foods', 'Warm blooded', 'Weeds', 'Wildlife biologist', 'Zoologists']
    
    set_scores = {}

    total_keywords = len(keywords)
    for i, keyword in enumerate(keywords, 1):
       articles = extract_articles(keyword)
       tfidf = extract_tfidf_from_articles(keyword, articles)
       
       set_scores[keyword] = tfidf
       print(f"Progress: {i}/{total_keywords} - keyword: {keyword}, tfidf: {tfidf}")
       
       if setno not in all_articles:
           all_articles[setno] = {}
       all_articles[setno][keyword] = articles
    
    weighted_keywords[setno] = set_scores
  
  return weighted_keywords, all_articles

weighted_keywords, all_articles = extract_weighted_keywords([3], df, prompts)

    
   
   


[nltk_data] Downloading package punkt to /Users/josephtey/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


All links:  ['https://en.wikipedia.org/wiki/China', 'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories/china-travel-advisory.html', 'https://english.www.gov.cn/', 'https://www.reuters.com/world/china/', 'https://m.airchina.com.cn/', 'https://www.state.gov/countries-areas/china/', 'https://www.stats.gov.cn/english/', 'https://www.china-airlines.com/us/en', 'https://www.fmprc.gov.cn/mfa_eng/', 'https://obamawhitehouse.archives.gov/the-press-office/2014/11/11/us-china-joint-announcement-climate-change', 'https://us.ceair.com/', 'https://www.chinamobileltd.com/', 'https://www.nottingham.edu.cn/en/index.aspx', 'https://www.census.gov/foreign-trade/balance/c5700.html', 'https://www.cia.gov/the-world-factbook/countries/china/', 'https://obamawhitehouse.archives.gov/the-press-office/2015/09/25/us-china-joint-presidential-statement-climate-change', 'https://www.cnsa.gov.cn/english/', 'https://www.scmp.com/asia', 'http://english.customs.gov.cn/', 'https://www.theguardi

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 5/94 - keyword: specialist, tfidf: 0.07720580872908468
All links:  ['https://www.eatsonponce.net/', 'https://www.ubereats.com/', 'https://www.dictionary.com/browse/eats', 'https://www.seriouseats.com/', 'https://www.youtube.com/@KatinaEatsKilos', 'https://eatsparkcity.org/', 'https://www.eatsrestaurantsf.com/', 'https://joanne-eatswellwithothers.com/', 'https://www.facebook.com/katinaeatskilos/', 'https://www.howsweeteats.com/', 'https://www.instagram.com/eats.and.the.city/', 'https://theeatsshow.us.messefrankfurt.com/us/en.html', 'https://www.reddit.com/r/seriouseats/', 'https://www.recipetineats.com/category/quick-and-easy/', 'https://littleeatsandthings.com/', 'https://www.recipetineats.com/', 'https://www.instagram.com/joelle_eats_cake/?hl=en', 'https://manila-eats.com/', 'https://www.instagram.com/dajeneats/?hl=en', 'https://www.tiktok.com/@how.kev.eats?lang=en']
Progress: 6/94 - keyword: eats, tfidf: 0.08731380107729611
All links:  ['https://www.bamboohr.com/', 'https:/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Progress: 87/94 - keyword: resource dependency, tfidf: 0.04786770143877246
All links:  ['https://www.sciencedirect.com/topics/biochemistry-genetics-and-molecular-biology/species-coexistence', 'https://besjournals.onlinelibrary.wiley.com/doi/10.1111/1365-2656.12879', 'https://www.journals.uchicago.edu/doi/10.1086/282505', 'https://www.jstor.org/stable/2459090', 'https://www.nature.com/articles/s41467-023-43279-y', 'https://www.cell.com/trends/ecology-evolution/fulltext/S0169-5347(20)30339-6', 'https://esajournals.onlinelibrary.wiley.com/doi/abs/10.1890/0012-9658(2001)082[0175%3AAVBIOH]2.0.CO%3B2', 'https://aslopubs.onlinelibrary.wiley.com/doi/abs/10.4319/lo.1978.23.6.1126', 'https://esajournals.onlinelibrary.wiley.com/doi/10.1890/10-0154.1', 'https://scholarworks.umt.edu/cgi/viewcontent.cgi?article=1034&context=wildbio_pubs', 'https://en.wikipedia.org/wiki/Coexistence_theory', 'https://nph.onlinelibrary.wiley.com/doi/abs/10.1111/nph.16865', 'https://academic.oup.com/jmammal/article/83/1

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 88/94 - keyword: coexisting species, tfidf: 0.03709276424597342
All links:  ['https://en.wikipedia.org/wiki/Predation', 'https://www.montecookgames.com/store/product/predation/', 'https://www.sciencedirect.com/topics/agricultural-and-biological-sciences/predation', 'https://www.reddit.com/r/mtgrules/comments/18hw122/ezuris_predation_and_ward/', 'https://www.fisheries.noaa.gov/feature-story/first-direct-evidence-killer-whale-predation-bowhead-whales-us-pacific-arctic', 'https://www.merriam-webster.com/dictionary/predation', 'https://www.nature.com/scitable/knowledge/library/predation-herbivory-and-parasitism-13261134/', 'https://www.ecologyandsociety.org/vol6/iss1/art11/', 'https://pubs.nmsu.edu/_circulars/CR688/', 'https://uapress.arizona.edu/book/landscapes-of-movement-and-predation', 'https://oceanservice.noaa.gov/education/tutorial_corals/coral08_naturalthreats.html', 'https://www.gwct.org.uk/game/research/predation-control/', 'https://www.britannica.com/science/predation'

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Progress: 93/94 - keyword: confinement, tfidf: 0.09750227742822383
All links:  ['https://reliance.com/', 'https://www.reliancetelephone.com/', 'https://www.ril.com/', 'https://reliancebank.bank/', 'https://www.relianceorthodontics.com/', 'https://reliancebankmn.com/', 'https://www.reliancestandard.com/home/', 'https://www.reliancecu.org/', 'https://www.youtube.com/channel/UCIMXKin1fXXCeq2UJePJEog/videos', 'https://en.wikipedia.org/wiki/Reliance_Industries', 'https://www.greif.com/reliance-products/', 'https://ilsr.org/', 'https://www.theparisreview.org/blog/2020/01/15/the-myth-of-self-reliance/', 'https://smartirb.org/reliance/', 'https://getreliancehealth.com/', 'https://research.rutgers.edu/faculty-staff/compliance/human-research-protection/reliance', 'https://www.research.uky.edu/office-research-integrity/single-irb-reliance', 'https://www.irs.gov/newsroom/general-overview-of-taxpayer-reliance-on-guidance-published-in-the-internal-revenue-bulletin-and-faqs', 'https://irbreliance.uco

In [140]:
from utils.feature_extractor import FeatureExtractor

feature_extractor = FeatureExtractor(prompts, only_prompts, model, doc2vec_model, ngram_results, weighted_keywords)

train model

In [142]:
from numpy import hstack

# run this for everythting ... so i can just do it once, and then save it as a csv to re-load ... 
def add_features_to_df(df, feature_extractor, add_word2vec=False, add_doc2vec=False, add_pos=False, add_prompt_overlap=False, add_weighted_keywords=False, add_lexical_overlap=False, add_stylistic_features=False, add_logical_operators=False, add_temporal_features=False):
    if add_word2vec:
        df['word2vec_features'] = None
    if add_doc2vec:
        df['doc2vec_features'] = None
    if add_pos:
        df['pos_features'] = None
    if add_prompt_overlap:
        df['prompt_overlap_features'] = None
    if add_weighted_keywords:
        df['weighted_keywords_features'] = None
    if add_lexical_overlap:
        df['lexical_overlap_features'] = None
    if add_stylistic_features:
        df['stylistic_features'] = None
    if add_logical_operators:
        df['logical_operators_features'] = None
    if add_temporal_features:
        df['temporal_features'] = None

    total_items = len(df)
    print(f"Processing {total_items} items...")

    # Iterate over each row in the training DataFrame
    num = 0
    for index, row in df.iterrows():
        # Extract features using the feature_extractor object
        if add_word2vec:
            word2vec_features = feature_extractor.word2vec(row['EssayText'])
            df.at[index, 'word2vec_features'] = word2vec_features
        if add_doc2vec:
            doc2vec_features = feature_extractor.doc2vec(row['EssayText'])
            df.at[index, 'doc2vec_features'] = doc2vec_features
        if add_pos:
            pos_features = feature_extractor.pos(row['EssaySet'], row['EssayText'])
            df.at[index, 'pos_features'] = pos_features
        if add_prompt_overlap:
            prompt_overlap_features = feature_extractor.prompt_overlap(row['EssaySet'], row['EssayText'])
            df.at[index, 'prompt_overlap_features'] = prompt_overlap_features
        if add_weighted_keywords:
            weighted_keywords_features = feature_extractor.weighted_domain_specific_keywords(row['EssaySet'], row['EssayText'])
            df.at[index, 'weighted_keywords_features'] = weighted_keywords_features
        if add_lexical_overlap:
            lexical_overlap_features = feature_extractor.lexical_overlap(row['EssaySet'], row['EssayText'])
            df.at[index, 'lexical_overlap_features'] = lexical_overlap_features
        if add_stylistic_features:
            stylistic_features = feature_extractor.stylistic_features(row['EssayText'])
            df.at[index, 'stylistic_features'] = stylistic_features
        if add_logical_operators:
            logical_operators_features = feature_extractor.logical_operators(row['EssayText'])
            df.at[index, 'logical_operators_features'] = logical_operators_features
        if add_temporal_features:
            temporal_features = feature_extractor.temporal_features(row['EssayText'])
            df.at[index, 'temporal_features'] = temporal_features

        # Update after every 1000 items
        if (num) % 1000 == 0:
            print(f"Processed {num} items.")
        num += 1

    return df

In [144]:
essay_set = 3

In [145]:
essay_set_df = df[df['EssaySet'] == essay_set].copy(deep=True)

In [161]:
# Save the updated DataFrame to a CSV file
import os
from datetime import datetime

# Execute the function to add features to the DataFrame and display the updated DataFrame
add_features_to_df(essay_set_df, feature_extractor, add_logical_operators=True)

# Create the folder if it doesn't exist
if not os.path.exists('updated_features'):
    os.makedirs('updated_features')

# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save the updated DataFrame to a CSV file with the timestamp
essay_set_df.to_csv(f'updated_features/updated_features_{timestamp}.csv', index=False)


Processing 3152 items...
Processed 0 items.
Processed 1000 items.
Processed 2000 items.
Processed 3000 items.


In [162]:
essay_set_df

,Id,DataSet,EssaySet,Score1,Score2,EssayText,Word_Count,word2vec_features,doc2vec_features,pos_features,prompt_overlap_features,weighted_keywords_features,lexical_overlap_features,stylistic_features,logical_operators_features,temporal_features
4917,4918,Train,3,1,1,China's panda and Australia's koala are two an...,50,"[0.016498972, 0.031992994, -0.05212467, 0.1381...","[-0.12533814, -0.19425772, -0.11712208, 0.1578...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, ...",[0.5],"[0.5367291916595557, 0.4403533819564889, 0.0, ...",[0.05],"[57, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2, 1, 0.03508771929824561, 1, 57, 3]","[8, 0, 0, 10, 2, 0, 0]"
4918,4919,Train,3,1,2,Pandas and koalas are similar because they are...,57,"[0.024004769, 0.044829894, -0.07149012, 0.1326...","[-0.054628838, -0.020331863, -0.1796877, 0.262...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, ...",[0.54],"[0.0, 0.0, 0.6039808046123907, 0.0, 0.0, 0.0, ...",[0.06],"[63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[5, 2, 0.07936507936507936, 1, 63, 2]","[6, 1, 0, 9, 0, 1, 0]"
4919,4920,Train,3,1,1,Pandas in China and Koalas in Australia are si...,33,"[0.026684571, 0.032433067, -0.060349528, 0.149...","[0.030490095, -0.09904233, 0.011396943, 0.2152...","[1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, ...",[0.42],"[0.5367291916595557, 0.4403533819564889, 0.0, ...",[0.03],"[36, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2, 1, 0.05555555555555555, 0, 36, 3]","[6, 0, 0, 5, 1, 0, 0]"
4920,4921,Train,3,2,1,Pandas in China only eat bamboo and Koalas in ...,32,"[0.006107875, 0.04953003, -0.058074407, 0.1578...","[-0.047954723, -0.00897786, -0.018569963, 0.12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[0.38],"[0.5367291916595557, 0.4403533819564889, 0.0, ...",[0.03],"[34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0.029411764705882353, 0, 34, 2]","[5, 1, 0, 4, 1, 1, 0]"
4921,4922,Train,3,0,0,Pandas in China and koalas from Australia are ...,27,"[0.0075033437, 0.01469695, -0.046362836, 0.130...","[0.11114751, -0.024824172, -0.093851134, 0.109...","[1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, ...",[0.5],"[0.5367291916595557, 0.4403533819564889, 0.0, ...",[0.03],"[29, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0.034482758620689655, 0, 29, 2]","[4, 0, 0, 4, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8064,8065,Priva,3,2,2,China's panda are similar to koalas because Th...,14,"[-0.020585494, 0.07710405, -0.07188554, 0.1588...","[0.06200168, -0.13997632, -0.08725603, 0.11112...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.33],"[0.0, 0.4403533819564889, 0.0, 0.0, 0.0, 0.0, ...",[0.02],"[16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 1, 0.0625, 0, 16, 1]","[3, 0, 0, 3, 0, 0, 0]"
8065,8066,Priva,3,1,1,Pandas from China are similar to koalas in Aus...,25,"[0.009386699, 0.0152493315, -0.07514191, 0.119...","[0.07324394, -0.050994035, -0.07131314, 0.1118...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[0.5],"[0.5367291916595557, 0.4403533819564889, 0.0, ...",[0.03],"[27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[0, 0, 0.0, 0, 27, 2]","[2, 0, 0, 2, 0, 0, 0]"
8066,8067,Priva,3,1,1,Koalas and panda are both similar in that they...,49,"[0.019533284, 0.06098972, -0.046831258, 0.1476...","[0.051570136, -0.08533205, -0.05561295, 0.2763...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, ...",[0.42],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.58742134...",[0.05],"[55, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[2, 1, 0.03636363636363636, 0, 55, 4]","[7, 0, 0, 9, 0, 0, 0]"
8067,8068,Priva,3,1,0,Pandas in China and koalas in Australia are si...,53,"[0.013115416, 0.043629017, -0.0131985275, 0.14...","[-0.107658185, -0.012414971, -0.031618986, 0.2...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",[0.5],"[0.5367291916595557, 0.4403533819564889, 0.0, ...",[0.05],"[59, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[3, 1, 0.05084745762711865, 1, 59, 3]","[9, 0, 0, 11, 0, 0, 0]"


In [163]:
features = ['word2vec_features', 'doc2vec_features', 'pos_features', 'prompt_overlap_features', 'weighted_keywords_features', 'lexical_overlap_features', 'stylistic_features', 'logical_operators_features', 'temporal_features']

In [164]:
# prepare data set 
import numpy as np

training_data = essay_set_df[(essay_set_df['DataSet'] == 'Train')].copy(deep=True)
training_data = training_data.dropna(subset=['word2vec_features', 'doc2vec_features'])
X_train = np.vstack(training_data[features].apply(lambda x: np.hstack(x), axis=1).values)
y_train = training_data['Score1']

In [165]:
# train model; hyper-parameter search
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV

param_dist = {'n_estimators': randint(50,500),
              'max_depth': randint(1,20)}
clf = RandomForestClassifier()
rand_search = RandomizedSearchCV(clf, 
                                param_distributions = param_dist, 
                                n_iter=5, 
                                cv=5)
                                

rand_search.fit(X_train, y_train)
best_clf = rand_search.best_estimator_

In [ ]:
# faster, more estimates
clf = RandomForestClassifier(max_depth=100, n_estimators=100)
clf.fit(X_train, y_train)

best_clf = clf

test model

In [ ]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix

val_set = essay_set_df[(essay_set_df['DataSet'] == 'Priva')].copy(deep=True)
val_set = val_set.dropna(subset=['word2vec_features', 'doc2vec_features'])

X_val = np.vstack(val_set[features].apply(lambda x: np.hstack(x), axis=1).values)
y_val = val_set['Score1']

y_pred = best_clf.predict(X_val)

conf_matrix = confusion_matrix(y_val, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

kappa_score = cohen_kappa_score(y_val, y_pred, weights='quadratic')
print(f"Cohen Kappa Score (Weighted): {kappa_score}")


Confusion Matrix:
[[ 24 123   2]
 [ 14 338   8]
 [  6  98  18]]
Cohen Kappa Score (Weighted): 0.1950373198372678
